In [122]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import sklearn.metrics as skm
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder,MinMaxScaler,StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import numpy as np

In [123]:
def group_pivot(labelgroup, yvalue, dataset, SZOnly = False):
    if SZOnly:
        dataset = dataset.loc[dataset.Diagnosis == 0]
    
    grouped = (dataset.groupby([labelgroup])[yvalue].value_counts(normalize = True).rename('percentage').reset_index())
    pivot = pd.pivot_table(grouped, index = labelgroup, columns = yvalue, values = 'percentage', aggfun = 'sum')
    return pivot

In [124]:
data = pd.read_csv('/Users/dahaixing/Documents/Coursework/DeepLearning/MLCW/MachineLearningCW/MS4S16_Dataset.csv')

In [125]:
num_var = [
 'Anhedonia',
 'Apathy',
 'Appetite',
 'Concentration',
 'Content',
 'Delusion',
 'Dep_Mood',
 'Focus',
 'Hallucination',
 'Intrusive_Thoughts',
 'Participant',
 'Passive',
 'Pregnant',
 'Psychomotor',
 'Rumination',
 'Sleep',
 'Stress',
 'Suspicious',
 'Tension',
 'Tired',
 'Unusual_Thought',
 'Withdrawal']

In [126]:
def data_impu(data):
    data.fillna(data.mean(), inplace=True)
    mean_tired = data['Tired'].replace(['NaN','inf'], pd.np.nan).astype(float).mean()
    mean_tired = data['Tired'].replace([pd.np.nan, pd.np.inf], pd.np.nan).mean()
    data['Tired'].replace(['NaN','inf'], mean_tired)
    data.replace([pd.np.nan, pd.np.inf], [mean_tired, mean_tired], inplace=True)
    return data

def data_remove(data):
    data.fillna(data.mean(), inplace=True)
    data = data.replace([np.inf, -np.inf], np.nan)
    data = data.dropna()

    # Create a copy of the dataframe
    data_standardized = data.copy()

    # Initialize the StandardScaler
    scaler = StandardScaler()

    # Fit and transform the selected columns
    data_standardized[num_var] = scaler.fit_transform(data[num_var])

    return data_standardized
    

In [127]:
data.describe()

,Diagnosis,Anhedonia,Apathy,Appetite,Concentration,Content,Delusion,Dep_Mood,Focus,Hallucination,...,Pregnant,Psychomotor,Rumination,Sleep,Stress,Suspicious,Tension,Tired,Unusual_Thought,Withdrawal
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,...,2762.000000,5000.000000,5000.000000,5000.000000,5000.000000,2127.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.505800,6.502860,2.478849,27.070029,6.519924,0.279407,2.637389,5.727062,6.519924,65.073832,...,0.101376,4.680583,5.685816,7.011266,4.916418,2.754909,4.920667,inf,2.480266,3.958121
std,0.500016,1.488151,1.730810,14.202618,1.474846,0.834494,1.440347,3.284501,1.474846,223.943469,...,0.301880,1.482692,2.161891,1.410841,2.220262,1.496126,1.962342,NaN,1.410475,1.469444
min,0.000000,1.098854,-3.211011,0.141074,1.299964,0.000187,-2.127037,0.000000,1.299964,0.027350,...,0.000000,-0.024974,-0.409032,2.144726,-3.257788,-2.346238,-2.183456,0.366650,-1.981307,-0.825919
25%,0.000000,5.495361,1.265128,16.724108,5.528181,0.018655,1.629919,4.678095,5.528181,4.113962,...,0.000000,3.697870,4.042552,6.058402,3.443683,1.703462,3.565482,4.491580,1.486439,2.969534
50%,1.000000,6.485527,2.427409,25.165292,6.498042,0.064259,2.558146,6.752196,6.498042,12.764604,...,0.000000,4.720156,5.521805,6.980519,5.096416,2.735108,5.247353,5.513508,2.388994,3.962131
75%,1.000000,7.489218,3.642059,35.447666,7.519759,0.215773,3.588012,8.045706,7.519759,41.814204,...,0.000000,5.682627,7.276673,7.977138,6.531673,3.725759,6.385145,6.569176,3.426667,4.972302
max,1.000000,11.603140,8.803433,113.438734,11.649649,21.001327,8.978785,12.003550,11.649649,6287.163151,...,1.000000,10.171540,12.009666,11.920312,11.970952,8.212275,9.622076,inf,8.066822,9.022207


In [128]:
features = [
 'Anhedonia',
 'Apathy',
 'Appetite',
 'Concentration',
 'Content',
 'Delay',
 'Delusion',
 'Dep_Mood',
 'Focus',
 'Hallucination',
 'Housing',
 'Intrusive_Thoughts',
 'Participant',
 'Passive',
 'Pregnant',
 'Psychomotor',
 'Race',
 'Rumination',
 'Sex',
 'Sleep',
 'Stress',
 'Suspicious',
 'Tension',
 'Tired',
 'Unusual_Thought',
 'Withdrawal']

In [129]:
#split the training and testing datasets
y = data['Diagnosis']
X = data[features]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
#X_test_save = X_test


train = pd.concat([X_train,y_train], axis= 1)
test = pd.concat([X_test,y_test], axis= 1)
train = data_remove(train)
test = data_remove(test)


/var/folders/xr/9g352cv10lv4vkg0vccmw9lc0000gn/T/ipykernel_74445/3066994613.py:10: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  data.fillna(data.mean(), inplace=True)
/var/folders/xr/9g352cv10lv4vkg0vccmw9lc0000gn/T/ipykernel_74445/3066994613.py:10: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  data.fillna(data.mean(), inplace=True)


In [130]:
train.describe()

,Anhedonia,Apathy,Appetite,Concentration,Content,Delusion,Dep_Mood,Focus,Hallucination,Intrusive_Thoughts,...,Psychomotor,Rumination,Sleep,Stress,Suspicious,Tension,Tired,Unusual_Thought,Withdrawal,Diagnosis
count,3.453000e+03,3.453000e+03,3.453000e+03,3.453000e+03,3.453000e+03,3.453000e+03,3.453000e+03,3.453000e+03,3.453000e+03,3.453000e+03,...,3.453000e+03,3.453000e+03,3.453000e+03,3.453000e+03,3.453000e+03,3.453000e+03,3.453000e+03,3.453000e+03,3.453000e+03,3453.000000
mean,-2.157749e-16,-2.271890e-16,8.333907e-17,-4.582363e-16,1.083537e-17,-1.647811e-16,-1.222194e-16,-4.582363e-16,2.996605e-17,-3.899767e-16,...,-7.438462e-17,4.086251e-16,-4.157129e-16,4.555274e-16,-2.316582e-16,8.867637e-17,-2.292146e-16,-6.533372e-17,1.093182e-18,0.506226
std,1.000145e+00,1.000145e+00,1.000145e+00,1.000145e+00,1.000145e+00,1.000145e+00,1.000145e+00,1.000145e+00,1.000145e+00,1.000145e+00,...,1.000145e+00,1.000145e+00,1.000145e+00,1.000145e+00,1.000145e+00,1.000145e+00,1.000145e+00,1.000145e+00,1.000145e+00,0.500034
min,-3.643784e+00,-3.293595e+00,-1.909822e+00,-3.317602e+00,-3.219297e-01,-2.873520e+00,-1.726255e+00,-3.317602e+00,-2.826740e-01,-3.288288e+00,...,-3.088765e+00,-2.837466e+00,-3.445138e+00,-3.178598e+00,-5.188993e+00,-2.876826e+00,-3.225170e+00,-3.169919e+00,-3.265643e+00,0.000000
25%,-6.813408e-01,-6.981994e-01,-7.296684e-01,-6.747620e-01,-3.008009e-01,-7.031888e-01,-3.489342e-01,-6.747620e-01,-2.655397e-01,-6.347742e-01,...,-6.697896e-01,-7.577776e-01,-6.738283e-01,-6.670958e-01,-1.363541e-03,-6.946101e-01,-6.980923e-01,-7.020191e-01,-6.666874e-01,0.000000
50%,-1.636020e-02,-2.885746e-02,-1.300228e-01,-2.811172e-02,-2.466557e-01,-5.869392e-02,3.108405e-01,-2.811172e-02,-2.295686e-01,-1.303176e-03,...,2.032149e-02,-7.404313e-02,-2.003069e-02,7.991253e-02,-1.363541e-03,1.549147e-01,-3.130752e-02,-6.377571e-02,-8.408409e-03,1.000000
75%,6.656224e-01,6.621856e-01,5.997694e-01,6.799822e-01,-7.157951e-02,6.607613e-01,7.122360e-01,6.799822e-01,-1.022276e-01,5.885606e-01,...,6.766168e-01,7.347916e-01,6.812309e-01,7.298914e-01,-1.363541e-03,7.494450e-01,6.782228e-01,6.620708e-01,6.916493e-01,1.000000
max,3.432298e+00,3.649743e+00,5.103752e+00,3.457625e+00,2.456202e+01,3.393701e+00,1.912724e+00,3.457625e+00,2.606179e+01,3.478570e+00,...,3.717284e+00,2.861600e+00,3.476470e+00,2.842187e+00,5.582565e+00,2.378891e+00,3.977292e+00,3.968454e+00,3.461964e+00,1.000000


In [131]:

categories = ['Sex','Race','Housing','Delay']

def onehot(data, categories = categories):
    ordinalencoder = OneHotEncoder()
    onehot = ordinalencoder.fit_transform(data[categories])
    columns = []
    for i, values in enumerate(ordinalencoder.categories_):
        for j in values:
            columns. append(categories[i]+'-'+j)
    return pd.DataFrame(onehot.toarray(), columns = columns)


In [132]:
#X_train = X_train.join(onehot(X_train))
#X_test = X_test.join(onehot(X_test))
X_train = pd.get_dummies(train[features], columns = categories)
X_test = pd.get_dummies(test[features], columns = categories)
y_train = train[['Diagnosis']]
y_test = test[['Diagnosis']]


,Diagnosis
2858,0
1559,1
1441,1
2179,0
1390,1


In [161]:
X_train.head()

,Anhedonia,Apathy,Appetite,Concentration,Content,Delusion,Dep_Mood,Focus,Hallucination,Intrusive_Thoughts,...,Sex_Female,Sex_Male,Race_Asian,Race_Black,Race_Hispanic,Race_White,Housing_Stable,Housing_Unstable,Delay_No,Delay_Yes
2858,-0.149288,0.343902,-1.844159,1.448343,0.687741,-0.220993,0.301635,1.448343,-0.276452,2.062382,...,1,0,0,1,0,0,1,0,0,1
1559,-0.501669,0.582127,0.851270,1.451164,-0.209577,-0.398124,0.750397,1.451164,-0.251090,-1.604057,...,0,1,0,0,0,1,1,0,1,0
1441,0.079694,-1.013840,0.596077,-0.923915,-0.293589,2.801352,-1.726255,-0.923915,8.702490,-2.515437,...,0,1,0,1,0,0,1,0,0,1
2179,0.979477,0.321785,-1.801085,-0.836934,-0.118058,-1.386109,0.742591,-0.836934,-0.280016,1.536706,...,1,0,0,1,0,0,1,0,1,0
1390,-1.353747,-0.402364,-0.923698,2.169370,-0.309835,0.003844,-1.726255,2.169370,-0.270162,1.112716,...,1,0,0,0,0,1,1,0,0,1


In [133]:
#Logistc regression training 

LGmodel = LogisticRegression(penalty='elasticnet', max_iter= 1000, solver= 'saga', l1_ratio=1)

LGmodel.fit(X_train, y_train)

LG_test_predict = LGmodel.predict(X_test)

/Users/dahaixing/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [134]:
print('LG test accuracy:', skm.accuracy_score(y_test, LG_test_predict))

LG test accuracy: 0.927027027027027


In [135]:
from sklearn.metrics import classification_report
print(classification_report(y_test, LG_test_predict, labels=[1, 0]))

              precision    recall  f1-score   support

           1       0.93      0.93      0.93       741
           0       0.93      0.93      0.93       739

    accuracy                           0.93      1480
   macro avg       0.93      0.93      0.93      1480
weighted avg       0.93      0.93      0.93      1480



In [136]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, LG_test_predict, labels=[1, 0])
print(confusion_matrix)

[[688  53]
 [ 55 684]]


In [137]:
import fairlearn
from fairlearn.metrics import MetricFrame

In [138]:
def race_fp(truelabels, predictions):
  sesitive = test.Race
  fmetrics = MetricFrame(metrics= fairlearn.metrics.false_positive_rate, 
                         y_true=truelabels, 
                         y_pred=predictions,
                         sensitive_features=sesitive)
  results = pd.DataFrame([fmetrics.by_group, fmetrics.by_group/fmetrics.by_group.White], 
                         index= ['FPR', 'FPR Parity'])
  return results

In [139]:
race_fp(y_test, LG_test_predict)

Race,Asian,Black,Hispanic,White
FPR,0.029197,0.131313,0.085271,0.050909
FPR Parity,0.573514,2.579365,1.674972,1.000000


In [140]:
def raceNsex_fp(truelabels, predictions):
  sesitive = pd.DataFrame(np.stack([test.Race, test.Sex], axis = 1),
                          columns = ['Race','Sex']) 
  fmetrics = MetricFrame(metrics= fairlearn.metrics.false_positive_rate, 
                         y_true=truelabels, 
                         y_pred=predictions,
                         sensitive_features=sesitive)
  results = pd.DataFrame([fmetrics.by_group, fmetrics.by_group/fmetrics.by_group.White.Male], 
                         index= ['FPR', 'FPR Parity'])
  return results

In [141]:
raceNsex_fp(y_test, LG_test_predict)

Race           Asian               Black            Hispanic            White  \
Sex           Female      Male    Female      Male    Female      Male Female   
FPR         0.025316  0.034483  0.132075  0.128205  0.090909  0.076923    0.0   
FPR Parity  0.193490  0.263547  1.009434  0.979853  0.694805  0.587912    0.0   

Race                  
Sex             Male  
FPR         0.130841  
FPR Parity  1.000000

In [142]:
def race_fn(truelabels, predictions):
  sesitive = test.Race
  fmetrics = MetricFrame(metrics= fairlearn.metrics.false_negative_rate, 
                         y_true=truelabels, 
                         y_pred=predictions,
                         sensitive_features=sesitive)
  results = pd.DataFrame([fmetrics.by_group, fmetrics.by_group/fmetrics.by_group.White], 
                         index= ['FNR', 'FNR Parity'])
  return results

In [143]:
race_fn(y_test, LG_test_predict)

Race,Asian,Black,Hispanic,White
FNR,0.050000,0.069182,0.085106,0.076305
FNR Parity,0.655263,0.906653,1.115342,1.000000


In [144]:
def raceNsex_fn(truelabels, predictions):
  sesitive = pd.DataFrame(np.stack([test.Race, test.Sex], axis = 1),
                          columns = ['Race','Sex']) 
  fmetrics = MetricFrame(metrics= fairlearn.metrics.false_negative_rate, 
                         y_true=truelabels, 
                         y_pred=predictions,
                         sensitive_features=sesitive)
  results = pd.DataFrame([fmetrics.by_group, fmetrics.by_group/fmetrics.by_group.White.Male], 
                         index= ['FNR', 'FNR Parity'])
  return results

In [145]:
raceNsex_fn(y_test, LG_test_predict)

Race           Asian               Black            Hispanic            \
Sex           Female      Male    Female      Male    Female      Male   
FNR         0.046512  0.054054  0.121495  0.042654  0.142857  0.022222   
FNR Parity  2.511628  2.918919  6.560748  2.303318  7.714286  1.200000   

Race           White            
Sex           Female      Male  
FNR         0.120567  0.018519  
FNR Parity  6.510638  1.000000

Model 2: Decision trees

In [146]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [147]:
DT_model = DecisionTreeClassifier()
DT_model = DT_model.fit(X_train, y_train)

In [148]:
DT_test_predictions = DT_model.predict(X_test)

In [149]:
print('DT test accuracy:', skm.accuracy_score(y_test, DT_test_predictions))

DT test accuracy: 0.8547297297297297


Model SVM

In [150]:
from sklearn import svm
svm_model = svm.SVC(kernel='linear')
svm_model = svm_model.fit(X_train, y_train)
svm_test_predictions = svm_model.predict(X_test)

/Users/dahaixing/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [151]:
print('SVM test accuracy:', skm.accuracy_score(y_test, svm_test_predictions))

SVM test accuracy: 0.9283783783783783


Light GBM model

In [168]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

# Define the LightGBM model
model = lgb.LGBMClassifier()

# Train the model
model.fit(X_train, y_train)

# Get the predicted probabilities for the test set
y_probs = model.predict_proba(X_test)[:, 1]

# Create an array of threshold values
thresholds = np.arange(0, 1.01, 0.01)

# Initialize an empty list to store the AUC scores
scores = []

# Iterate over the threshold values
for t in thresholds:
    # Convert the predicted probabilities to class labels using the current threshold
    y_pred = (y_probs > t).astype(int)
    # Calculate the AUC score for the current threshold
    score = roc_auc_score(y_test, y_pred)
    # Append the score to the list
    scores.append(score)

# Find the threshold that corresponds to the highest AUC score
best_threshold = thresholds[np.argmax(scores)]

print(f'The optimal threshold is {best_threshold:.2f}')

/Users/dahaixing/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/dahaixing/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The optimal threshold is 0.59


In [169]:
import lightgbm as lgb

train_data = lgb.Dataset(X_train, label=y_train)
params = {'objective': 'binary', 'metric': 'binary_logloss'}
LGB_model = lgb.train(params, train_data)
# Predict on the test data
LGB_test_predictions = LGB_model.predict(X_test)

# Convert the predicted probabilities to binary labels
LGB_test_predictions = (LGB_test_predictions > 0.59).astype(int)

[LightGBM] [Info] Number of positive: 1748, number of negative: 1705
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000975 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5124
[LightGBM] [Info] Number of data points in the train set: 3453, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.506226 -> initscore=0.024907
[LightGBM] [Info] Start training from score 0.024907


In [172]:
print('LGB test accuracy:', skm.accuracy_score(y_test, LGB_test_predictions))

LGB test accuracy: 0.9432432432432433


In [173]:
raceNsex_fp(y_test, LGB_test_predictions)

Race           Asian               Black            Hispanic            \
Sex           Female      Male    Female      Male    Female      Male   
FPR         0.025316  0.068966  0.081761  0.076923  0.103896  0.096154   
FPR Parity  0.386980  1.054187  1.249775  1.175824  1.588126  1.469780   

Race           White            
Sex           Female      Male  
FPR         0.023810  0.065421  
FPR Parity  0.363946  1.000000

In [174]:
race_fp(y_test, LGB_test_predictions)

Race,Asian,Black,Hispanic,White
FPR,0.043796,0.080808,0.100775,0.04
FPR Parity,1.094891,2.020202,2.519380,1.00


In [175]:
print(classification_report(y_test, LGB_test_predictions, labels=[1, 0]))

              precision    recall  f1-score   support

           1       0.94      0.95      0.94       741
           0       0.95      0.94      0.94       739

    accuracy                           0.94      1480
   macro avg       0.94      0.94      0.94      1480
weighted avg       0.94      0.94      0.94      1480



We choose Light GBM as classification model, next we do cross-validation

In [157]:
cv_data = data
cv_data = data_remove(cv_data)




/var/folders/xr/9g352cv10lv4vkg0vccmw9lc0000gn/T/ipykernel_74445/3066994613.py:10: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  data.fillna(data.mean(), inplace=True)


In [158]:
y = cv_data[['Diagnosis']]

In [159]:

X = pd.get_dummies(cv_data[features], columns = categories)
X.describe()

,Anhedonia,Apathy,Appetite,Concentration,Content,Delusion,Dep_Mood,Focus,Hallucination,Intrusive_Thoughts,...,Sex_Female,Sex_Male,Race_Asian,Race_Black,Race_Hispanic,Race_White,Housing_Stable,Housing_Unstable,Delay_No,Delay_Yes
count,4.933000e+03,4.933000e+03,4.933000e+03,4.933000e+03,4.933000e+03,4.933000e+03,4.933000e+03,4.933000e+03,4.933000e+03,4.933000e+03,...,4933.000000,4933.000000,4933.000000,4933.000000,4933.000000,4933.000000,4933.000000,4933.000000,4933.000000,4933.000000
mean,1.669948e-17,-3.688740e-17,-1.426883e-17,4.170158e-17,-7.061552e-17,-4.951329e-19,-2.676250e-17,4.170158e-17,3.384346e-17,1.678050e-16,...,0.553010,0.446990,0.141091,0.347659,0.142307,0.368944,0.934523,0.065477,0.484898,0.515102
std,1.000101e+00,1.000101e+00,1.000101e+00,1.000101e+00,1.000101e+00,1.000101e+00,1.000101e+00,1.000101e+00,1.000101e+00,1.000101e+00,...,0.497232,0.497232,0.348150,0.476275,0.349400,0.482568,0.247391,0.247391,0.499823,0.499823
min,-3.629545e+00,-3.285503e+00,-1.898814e+00,-3.530735e+00,-3.353126e-01,-3.309404e+00,-1.730247e+00,-3.530735e+00,-2.929525e-01,-3.269200e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-6.794537e-01,-7.007022e-01,-7.323402e-01,-6.712355e-01,-3.132315e-01,-7.010843e-01,-3.242380e-01,-6.712355e-01,-2.743609e-01,-6.559756e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,-1.224186e-02,-2.722499e-02,-1.352612e-01,-1.641533e-02,-2.583529e-01,-5.435813e-02,3.153087e-01,-1.641533e-02,-2.350200e-01,-1.803323e-03,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
75%,6.617310e-01,6.699285e-01,5.933205e-01,6.779146e-01,-7.612008e-02,6.617795e-01,7.090571e-01,6.779146e-01,-1.025699e-01,5.820789e-01,...,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000
max,3.424339e+00,3.657223e+00,5.067951e+00,3.474823e+00,2.487855e+01,4.408723e+00,1.910582e+00,3.474823e+00,2.837740e+01,3.458203e+00,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [160]:

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

# Define the LightGBM parameters
params = {'objective': 'binary', 'metric': 'binary_logloss'}

# Create a KFold object
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize a list to store the accuracy scores
scores = []
